In [1]:
import time

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
from openpyxl import load_workbook

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

import twitter

In [2]:
CK1 = config['TWITTER']['CK1']
CS1 = config['TWITTER']['CS1']
ATK1 = config['TWITTER']['ATK1']
ATS1 = config['TWITTER']['ATS1']

# set to extended mode and sleep if exceed rate limit
api1 = twitter.Api(consumer_key=CK1,
                  consumer_secret=CS1,
                  access_token_key=ATK1,
                  access_token_secret=ATS1,
                  tweet_mode= 'extended',
                  sleep_on_rate_limit=True)

In [3]:
db_twitter = client["Twitter"]
collections_twitter = db_twitter.collection_names()

In [4]:
dic_collection = {}
for i in collections_twitter:
    dic_collection[i] = "{:,}".format(db_twitter[i].find({}).count())


for key in sorted(dic_collection):
    print("%s: %s" % (key, dic_collection[key]))

2018_W51_Twitter_Australia: 20,471
2018_W51_Twitter_Other: 8,978,243
2018_W52_Twitter_Australia: 38,065
2018_W52_Twitter_Other: 16,940,708
2019_W10_Twitter_Australia: 3,174
2019_W10_Twitter_Other: 876,985
2019_W1_Twitter_Australia: 40,880
2019_W1_Twitter_Other: 16,994,959
2019_W2_Twitter_Australia: 37,645
2019_W2_Twitter_Other: 14,474,627
2019_W3_Twitter_Australia: 52,348
2019_W4_Twitter_Australia: 59,625
2019_W5_Twitter_Australia: 61,617
2019_W6_Twitter_Australia: 61,435
2019_W7_Twitter_Australia: 28,017
2019_W8_Twitter_Australia: 60,035
2019_W8_Twitter_Other_Old: 31,057
2019_W9_Twitter_Australia: 58,316
2019_W9_Twitter_Other: 22,293,795
2019_W_Twitter_Other: 4
twitter-other: 201,459,531
twitter-richard-2014: 211,755
twitter-richard-2015: 198,258
twitter-richard-2016: 10,801
twitter-richard-2017: 21,797,263
twitter-richard-2018: 10,679,787


In [ ]:
for e in sorted(dic_collection)[::4]:
    if "Australia" in e:
        print("--------------\n" + e)
        c = 0
        # get the first "limit_number" records from "twitter-australia" collection
        for document in db_twitter[e].find({"truncated":True,"soda_checked_truncated_utc":{"$exists":False}}):
                c += 1
                try:
                    # get full_text using twitter api
                    full_text = api1.GetStatus(document["id_str"]).full_text
                    print(str(c) + "--------------\n" + "got full_text")
                    
                    t = int(time.time())
                    # update fields(add full_text if exist and change truncated value to false)
                    db_twitter[e].update({"_id": document["_id"]}, {"$set": {"text": full_text, "truncated":False , "soda_checked_truncated_utc": t }})
                except:
                    
                    t = int(time.time())
                    db_twitter[e].update({"_id": document["_id"]}, {"$set": {"soda_checked_truncated_utc": t}})
                    print(str(c) + "--------------\n" + "pass except")

        print("--------------\n" + "Done: get full_text of " + str(c) + " truncated tweets in collection " + e)
    